<a href="https://colab.research.google.com/github/Hou-jing/paper_record_public/blob/main/SPN4RE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!wget https://www.dropbox.com/s/6kks8s4i8tpg1dc/SPN4RE-main.zip?dl=0  -O SPN4RE-main.zip
!unzip -q SPN4RE-main.zip

In [ ]:
!sudo apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/bert-base-cased
!cd bert-base-cased/
!git lfs install
!pip install --upgrade google-cloud-storage

In [6]:
!pip install transformers

In [ ]:
from trainer.trainer import Trainer

In [10]:
import argparse, os, torch
import random
import numpy as np
from utils.data import build_data
from trainer.trainer import Trainer
from models.setpred4RE import SetPred4RE

print(torch.cuda.device_count())
print(torch.cuda.is_available())

def str2bool(v):
    return v.lower() in ('true')


def add_argument_group(name):
    arg = parser.add_argument_group(name)
    return arg


def get_args():
    args, unparsed = parser.parse_known_args()
    if len(unparsed) > 1:
        print("Unparsed args: {}".format(unparsed))
    return args, unparsed


def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    data_arg = add_argument_group('Data')
    
    # data_arg.add_argument('--dataset_name', type=str, default="NYT-exact")
    # data_arg.add_argument('--train_file', type=str, default="./data/NYT/exact_data/train.json")
    # data_arg.add_argument('--valid_file', type=str, default="./data/NYT/exact_data/valid.json")
    # data_arg.add_argument('--test_file', type=str, default="./data/NYT/exact_data/test.json")
    
    # data_arg.add_argument('--dataset_name', type=str, default="NYT-partial")
    # data_arg.add_argument('--train_file', type=str, default="./data/NYT/casrel_data/new_train.json")
    # data_arg.add_argument('--valid_file', type=str, default="./data/NYT/casrel_data/new_valid.json")
    # data_arg.add_argument('--test_file', type=str, default="./data/NYT/casrel_data/new_test.json")
    data_arg.add_argument('--dataset_name', type=str, default="WebNLG")
    data_arg.add_argument('--train_file', type=str, default="./data/WebNLG/clean_WebNLG/new_train.json")
    data_arg.add_argument('--valid_file', type=str, default="./data/WebNLG/clean_WebNLG/new_valid.json")
    data_arg.add_argument('--test_file', type=str, default="./data/WebNLG/clean_WebNLG/new_test.json")

    data_arg.add_argument('--generated_data_directory', type=str, default="./data/generated_data/")
    data_arg.add_argument('--generated_param_directory', type=str, default="./data/generated_data/model_param/")
    data_arg.add_argument('--bert_directory', type=str, default="bert-base-cased/")
    data_arg.add_argument("--partial", type=str2bool, default=False)
    learn_arg = add_argument_group('Learning')
    learn_arg.add_argument('--model_name', type=str, default="Set-Prediction-Networks")
    learn_arg.add_argument('--num_generated_triples', type=int, default=10)
    learn_arg.add_argument('--num_decoder_layers', type=int, default=3)
    learn_arg.add_argument('--matcher', type=str, default="avg", choices=['avg', 'min'])
    learn_arg.add_argument('--na_rel_coef', type=float, default=1)
    learn_arg.add_argument('--rel_loss_weight', type=float, default=1)
    learn_arg.add_argument('--head_ent_loss_weight', type=float, default=2)
    learn_arg.add_argument('--tail_ent_loss_weight', type=float, default=2)
    learn_arg.add_argument('--fix_bert_embeddings', type=str2bool, default=True)
    learn_arg.add_argument('--batch_size', type=int, default=8)#8
    learn_arg.add_argument('--max_epoch', type=int, default=50)
    learn_arg.add_argument('--gradient_accumulation_steps', type=int, default=1)
    learn_arg.add_argument('--decoder_lr', type=float, default=2e-5)
    learn_arg.add_argument('--encoder_lr', type=float, default=1e-5)
    learn_arg.add_argument('--lr_decay', type=float, default=0.01)
    learn_arg.add_argument('--weight_decay', type=float, default=1e-5)
    learn_arg.add_argument('--max_grad_norm', type=float, default=0)
    learn_arg.add_argument('--optimizer', type=str, default='AdamW', choices=['Adam', 'AdamW'])
    evaluation_arg = add_argument_group('Evaluation')
    evaluation_arg.add_argument('--n_best_size', type=int, default=100)
    evaluation_arg.add_argument('--max_span_length', type=int, default=12) #NYT webNLG 10
    misc_arg = add_argument_group('MISC')
    misc_arg.add_argument('--refresh', type=str2bool, default=False)
    misc_arg.add_argument('--use_gpu', type=str2bool, default=True)
    misc_arg.add_argument('--visible_gpu', type=int, default=1)
    misc_arg.add_argument('--random_seed', type=int, default=1)




    args, unparsed = get_args()
    os.environ["CUDA_VISIBLE_DEVICES"] = str(args.visible_gpu)
    for arg in vars(args):
        print(arg, ":",  getattr(args, arg))
    set_seed(args.random_seed)
    data = build_data(args)
    model = SetPred4RE(args, data.relational_alphabet.size())
    trainer = Trainer(model, data, args)
    trainer.train_model()


1
True
Unparsed args: ['-f', '/root/.local/share/jupyter/runtime/kernel-173aaee7-10f6-43db-9132-6b4d4ebaa6e7.json']
dataset_name : WebNLG
train_file : ./data/WebNLG/clean_WebNLG/new_train.json
valid_file : ./data/WebNLG/clean_WebNLG/new_valid.json
test_file : ./data/WebNLG/clean_WebNLG/new_test.json
generated_data_directory : ./data/generated_data/
generated_param_directory : ./data/generated_data/model_param/
bert_directory : bert-base-uncased/
partial : False
model_name : Set-Prediction-Networks
num_generated_triples : 10
num_decoder_layers : 3
matcher : avg
na_rel_coef : 1
rel_loss_weight : 1
head_ent_loss_weight : 2
tail_ent_loss_weight : 2
fix_bert_embeddings : True
batch_size : 8
max_epoch : 50
gradient_accumulation_steps : 1
decoder_lr : 2e-05
encoder_lr : 1e-05
lr_decay : 0.01
weight_decay : 1e-05
max_grad_norm : 0
optimizer : AdamW
n_best_size : 100
max_span_length : 12
refresh : False
use_gpu : True
visible_gpu : 1
random_seed : 1
Data setting is loaded from file:  ./data/gen

Some weights of the model checkpoint at bert-base-uncased/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in 

=== Epoch 0 train ===
     Instance: 800; loss: 7.0134
     Instance: 1600; loss: 4.8848
     Instance: 2400; loss: 3.9756
     Instance: 3200; loss: 3.4225
     Instance: 4000; loss: 3.0683
     Instance: 4800; loss: 2.8000
=== Epoch 0 Validation ===
--*--*--Num of Gold Triplet is 1--*--*--
gold_num =  196  pred_num =  129  right_num =  16  relation_right_num =  17  entity_right_num =  108
precision =  0.12403100775193798  recall =  0.08163265306122448  f1_value =  0.09846153846153848
rel_precision =  0.13178294573643412  rel_recall =  0.08673469387755102  rel_f1_value =  0.10461538461538462
ent_precision =  0.8372093023255814  ent_recall =  0.5510204081632653  ent_f1_value =  0.6646153846153845
--*--*--Num of Gold Triplet is 2--*--*--
gold_num =  224  pred_num =  107  right_num =  42  relation_right_num =  47  entity_right_num =  97
precision =  0.3925233644859813  recall =  0.1875  f1_value =  0.25377643504531716
rel_precision =  0.4392523364485981  rel_recall =  0.20982142857142858

这个结果是在bert-base-uncased条件下，得到的，而非论文中的bert-base-cased

In [12]:
eval_loader=data.test_loader
trainer.eval_model(eval_loader)

--*--*--Num of Gold Triplet is 1--*--*--
gold_num =  266  pred_num =  316  right_num =  230  relation_right_num =  231  entity_right_num =  296
precision =  0.7278481012658228  recall =  0.8646616541353384  f1_value =  0.7903780068728522
rel_precision =  0.7310126582278481  rel_recall =  0.868421052631579  rel_f1_value =  0.7938144329896907
ent_precision =  0.9367088607594937  ent_recall =  1.112781954887218  ent_f1_value =  1.0171821305841924
--*--*--Num of Gold Triplet is 2--*--*--
gold_num =  342  pred_num =  362  right_num =  301  relation_right_num =  305  entity_right_num =  349
precision =  0.8314917127071824  recall =  0.8801169590643275  f1_value =  0.8551136363636364
rel_precision =  0.8425414364640884  rel_recall =  0.8918128654970761  rel_f1_value =  0.8664772727272727
ent_precision =  0.9640883977900553  ent_recall =  1.0204678362573099  ent_f1_value =  0.9914772727272728
--*--*--Num of Gold Triplet is 3--*--*--
gold_num =  393  pred_num =  368  right_num =  338  relation_

{'f1': 0.8630050505050505,
 'precision': 0.8668357641090678,
 'recall': 0.8592080452545569}